# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colonia,50.9333,6.9500,3.31,92,100,3.60,DE,1704566238
1,1,waitangi,-43.9535,-176.5597,13.23,92,9,0.89,NZ,1704566341
2,2,hithadhoo,-0.6000,73.0833,27.44,79,100,6.11,MV,1704566342
3,3,olonkinbyen,70.9221,-8.7187,-6.18,85,98,5.12,SJ,1704566343
4,4,albany,42.6001,-73.9662,0.57,72,100,0.89,US,1704566217


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
190,190,pisco,-13.7000,-76.2167,26.03,61,0,4.12,PE,1704566472
274,274,kapa'a,22.0752,-159.3190,21.17,90,0,2.57,US,1704566271
292,292,taoudenni,22.6783,-3.9836,24.09,16,0,3.04,ML,1704566301
468,468,pamandzi,-12.7967,45.2794,26.88,94,0,0.51,YT,1704566665
487,487,kailua-kona,19.6406,-155.9956,25.33,65,0,2.06,US,1704566416
519,519,holualoa,19.6228,-155.9522,23.26,59,0,2.06,US,1704566699
544,544,aioun,16.6614,-9.6149,25.79,9,0,4.34,MR,1704566716


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
190,pisco,PE,-13.7000,-76.2167,61,
274,kapa'a,US,22.0752,-159.3190,90,
292,taoudenni,ML,22.6783,-3.9836,16,
468,pamandzi,YT,-12.7967,45.2794,94,
487,kailua-kona,US,19.6406,-155.9956,65,
519,holualoa,US,19.6228,-155.9522,59,
544,aioun,MR,16.6614,-9.6149,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
kapa'a - nearest hotel: Pono Kai Resort
taoudenni - nearest hotel: No hotel found
pamandzi - nearest hotel: Gite Le Madina
kailua-kona - nearest hotel: Kona Seaside Hotel
holualoa - nearest hotel: Kona Hotel
aioun - nearest hotel: فندق العيون


,City,Country,Lat,Lng,Humidity,Hotel Name
190,pisco,PE,-13.7000,-76.2167,61,La Portada
274,kapa'a,US,22.0752,-159.3190,90,Pono Kai Resort
292,taoudenni,ML,22.6783,-3.9836,16,No hotel found
468,pamandzi,YT,-12.7967,45.2794,94,Gite Le Madina
487,kailua-kona,US,19.6406,-155.9956,65,Kona Seaside Hotel
519,holualoa,US,19.6228,-155.9522,59,Kona Hotel
544,aioun,MR,16.6614,-9.6149,9,فندق العيون


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)